In [1]:
''# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os, sys
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))


# Suppress warnings 
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)



## Téléchargement des données 

In [2]:
app_train = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head() 

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
app_test = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\application_test.csv')
print('Training data shape: ', app_test.shape)
app_train.head()

Training data shape:  (48744, 121)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


application_train/application_test : les principales données de formation et de test avec des informations sur chaque demande de prêt chez Home Credit. Chaque prêt a sa propre ligne et est identifié par la caractéristique SK_ID_CURR. Les données de la demande de formation sont accompagnées de la CIBLE indiquant 0 : le prêt a été remboursé ou 1 : le prêt n'a pas été remboursé.

In [ ]:

app_previous = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\previous_application.csv')
print(f'app previous Shape: {app_previous.shape}')

installments_payments = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\installments_payments.csv')
print(f'installments payments Shape: {installments_payments.shape}')

sample_submission = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\sample_submission.csv')
print(f'sample submission Shape: {sample_submission.shape}')

bureau = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\bureau.csv')
print(f'bureau Shape: {bureau.shape}')

bureau_balance = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\bureau_balance.csv')
print(f'bureau balance Shape: {bureau_balance.shape}')

POS_CASH_balance = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\POS_CASH_balance.csv')
print(f'POS CASH balance Shape: {POS_CASH_balance.shape}')

credit_card_balance = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\credit_card_balance.csv')
print(f'credit card balance Shape: {credit_card_balance.shape}')


app previous Shape: (1670214, 37)
installments payments Shape: (13605401, 8)
sample submission Shape: (48744, 2)
bureau Shape: (1716428, 17)
bureau balance Shape: (27299925, 3)
POS CASH balance Shape: (10001358, 8)


In [ ]:
plt.figure(figsize=(12, 8))
app_train.dtypes.value_counts().plot.pie(explode=(0, 0, 0.2),autopct='%1.1f%%', colors=['#BD98FB','#B7BCFF', '#AEE7F2'],shadow=True)
plt.title("Types de données")
plt.show

# Analyse Exploratoire
La cible est ce qu'on nous demande de prévoir : soit un 0 pour le prêt a été remboursé à temps, soit un 1 indiquant que le client a eu des difficultés de paiement. Nous pouvons d'abord examiner le nombre de prêts entrant dans chaque catégorie

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
ax = sns.countplot(x=app_train['TARGET'], palette=['#BD98FB', '#AEE7F2'])
total = len(app_train['TARGET'])
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height() / total)
    x = p.get_x() + p.get_width() / 2 - 0.1
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', fontsize=12, color='black')
sns.despine()
plt.suptitle('Distribution of the Target Column', fontsize=12)
plt.title('0 = loan was repaid on time,    1 = client had payment difficulties.', fontsize=10)
plt.show()


## Valeurs manquantes

In [ ]:
print(app_train.columns)

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
# Nombre de classes uniques dans chaque colonne d'objet 
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_values = missing_values_table(app_train)
missing_values.head(20)

In [ ]:
missing_values.index

In [ ]:
def plot_feat(df:pd.DataFrame,feature:str,label_rotation=False,horizontal_layout=True):
    temp = df[feature].fillna('Unknown').value_counts()
    df1 = pd.DataFrame({feature: temp.index,'Number of contracts': temp.values})
    df1.sort_values(by=feature, inplace=True)

    # Calculate the percentage of each category with target=1
    cat_perc = df[[feature, 'TARGET']].fillna("Unknown").groupby([feature],as_index=False).mean()
    cat_perc['TARGET']*=100
    cat_perc.sort_values(by=feature, inplace=True)
    
    if(horizontal_layout):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,14))
    s = sns.barplot(ax=ax1, x = feature, y="Number of contracts",data=df1, color = 'blue')
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    ax1.set_title('Nombre de contrats par catégorie')
    sns.despine()

    s = sns.barplot(ax=ax2, x = feature, y='TARGET', order=cat_perc[feature], data=cat_perc, color = 'green')
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    ax2.set_ylabel('Pourcentage de la catégorie avec target valeur 1 [%]')
    ax2.set_title('Pourcentage de prêts en défaut de paiement')
    sns.despine()

    plt.suptitle(f"Contrats par catégorie de fonctionnalité '{feature}'")
    

In [ ]:
# Type de contrat
plot_feat(app_train, 'NAME_CONTRACT_TYPE')

On peut voir que la plupart des prêts que les clients prennent sont des prêts cash.
moin 6% des personnes ont contracté un prêt revolving.

In [ ]:
plot_feat(app_train, 'CODE_GENDER')


Le premier point à observer est qu'il y a 4 lignes dans la table application_train qui ont des genres "XNA".
Dans le subplot 1, nous constatons que pour l'ensemble de données donné, il y a plus de candidats féminins  que de candidats masculins.
Cependant, contrairement au nombre de candidates, nous constatons, à partir du deuxième subplot, que les candidats masculins ont tendance à être plus défaillants (10,14%) que les candidates féminines (7%).
Ainsi, on peut dire que les hommes ont plus tendance à ne pas payer leurs dettes que les femmes selon l'ensemble de données donné.

In [ ]:
# Optional: Remove 4 applications with XNA CODE_GENDER (train set)
app_train = app_train[app_train['CODE_GENDER'] != 'XNA']
app_test = app_test[app_test['CODE_GENDER'] != 'XNA']

Pour la variable 'NAME_INCOME_TYPE' :
La colonne 'NAME_INCOME_TYPE' prend la valeur 'Maternity leave' uniquement pour le jeu d'entrainement et pour seulement 5 emprunteurs que nous allons supprimé.

In [ ]:
app_train['NAME_INCOME_TYPE'].unique()

In [ ]:
app_test['NAME_INCOME_TYPE'].unique()

In [ ]:
# Supprime les individus dont 'NAME_INCOME_TYPE' vient d'un congé maternité
#app_train = app_train[app_train['NAME_INCOME_TYPE'] != 'Maternity leave']

 Pour la colonne NAME_FAMILY_STATUS, il y a seulement deux fois la valeur Unknown et uniquement pour le jeu d'entrainement. Les lignes correspondantes sont supprimées

In [ ]:
app_train['NAME_FAMILY_STATUS'].unique()

In [ ]:
app_test['NAME_FAMILY_STATUS'].unique()

In [ ]:
# Supprime les individus dont la statut familial est inconnu
app_train = app_train[app_train['NAME_FAMILY_STATUS'] != 'Unknown']

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 5))
plt.title("Les professions les plus représentées \n")
sns.countplot(data=app_train, x=app_train["OCCUPATION_TYPE"],  color="#b541a4")
plt.xticks(range(0,app_train["OCCUPATION_TYPE"].nunique()+1) ,app_train["OCCUPATION_TYPE"].unique(),rotation=90)
plt.grid()

### Alignement des données de train et de test
Il doit y avoir les mêmes caractéristiques (colonnes) dans les données de formation et de test. L'encodage à chaud a créé plus de colonnes dans les données d'apprentissage car il y avait des variables catégorielles avec des catégories non représentées dans les données de test. Pour supprimer les colonnes des données d'entraînement qui ne figurent pas dans les données de test, nous devons aligner les dataframes. Nous extrayons d'abord la colonne cible des données d'apprentissage (car cela ne figure pas dans les données de test, mais nous devons conserver ces informations). Lorsque nous faisons l'alignement, nous devons nous assurer de définir axis = 1 pour aligner les dataframes en fonction des colonnes et non des lignes

In [ ]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

### Traitement des outliers
### Anomalies
Numéros mal saisis, erreurs dans l'équipement de mesure ou mesures extrêmes.

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()


Il n'y a pas de valeurs aberrantes pour l'âge, que ce soit à l'extrémité supérieure ou inférieure.

In [ ]:
(app_train['DAYS_EMPLOYED'] / -365).describe()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram', color = "green", label='Train_DAYS_EMPLOYED')
app_test['DAYS_EMPLOYED'].plot.hist(color = "yellow", label='Test_DAYS_EMPLOYED')
plt.xlabel('Days Employment')
plt.legend()
app_test['DAYS_EMPLOYED'][app_test['DAYS_EMPLOYED'] > 200000]


sous-divisons les clients anormaux et voyons s'ils ont tendance à avoir des taux de défaut plus élevés ou plus faibles que le reste des clients

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

Il s'avère que les anomalies ont un taux de défaut plus faible. Nous allons remplir les valeurs anormales sans nombre (np.nan) puis créer une nouvelle colonne booléenne indiquant si la valeur était anormale ou non.

In [ ]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

plt.figure(figsize=(10, 8))
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram',color = "green", label='Train_DAYS_EMPLOYED')
app_test['DAYS_EMPLOYED'].plot.hist(color = "yellow", label='Test_DAYS_EMPLOYED')
plt.xlabel('Days Employment')
plt.legend()

print('Il y a %d anomalies dans les données de test sur %d entrées\n' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

In [ ]:
# Sauvegarde des données non encore encodées


app_train.to_csv('app_train_no_encoded_no_featureengineering.csv', index=False)
app_test.to_csv('app_test_no_encoded_no_featureengineering.csv', index=False)


### Effet de l'âge sur le remboursement

In [ ]:
# Find the correlation of the positive days since birth and target
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

À mesure que le client vieillit, il existe une relation linéaire négative avec la cible, ce qui signifie qu'à mesure que les clients vieillissent, ils ont tendance à rembourser leurs prêts à temps plus souvent.

### Répartition de l'âge

In [ ]:
# Plot the distribution of ages in years
plt.figure(figsize=(10, 8))
plt.hist(app_train['DAYS_BIRTH'] / 365, color = '#AEE7F2', edgecolor = 'black', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count')

In [ ]:
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target = 0', color = '#458b74')

# KDE plot of loans which were not repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target = 1',  color = '#dfa801')

# Labeling of plot
plt.xlabel('Age (years)')
plt.ylabel('Density')
plt.title('Distribution of Ages')
plt.legend()

La courbe cible == 1 s'incline vers l'extrémité la plus jeune de la plage. Bien qu'il ne s'agisse pas d'une corrélation significative (coefficient de corrélation de -0,07), cette variable sera probablement utile dans un modèle d'apprentissage automatique car elle affecte la cible. Regardons cette relation sous un autre angle : incapacité moyenne à rembourser les crédits par tranche d'âge.

Pour faire ce graphique, nous avons d'abord découpé la catégorie d'âge en tranches de 5 ans chacune. Ensuite, pour chaque bin, nous calculons la valeur moyenne de la cible, qui nous indique le ratio de prêts non remboursés dans chaque catégorie d'âge

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (10, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'], color = '#ae639b')

# Plot labeling
plt.xticks(rotation = 35)
plt.xlabel('Group age(years)')
plt.ylabel('Défaut de remboursement (%)')
plt.title("Défaut de remboursement par groupe d'âge")

 les jeunes demandeurs sont plus susceptibles de ne pas rembourser le prêt ! Le taux d'impayés est supérieur à 10 % pour les trois tranches d'âge les plus jeunes et inférieur à 5 % pour la tranche d'âge la plus élevée.

### Sources extérieures
Les 3 variables avec les plus fortes corrélations négatives avec Target sont EXT_SOURCE_1, EXT_SOURCE_2,  EXT_SOURCE_3

In [ ]:
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
# Heatmap of correlations
plt.figure(figsize = (10, 8))
#mask = np.triu(ext_data_corrs)

sns.heatmap(ext_data_corrs,  cmap='BrBG', vmin = -0.3, annot = True, vmax = 1)
plt.title('Correlation Heatmap')


Les trois caractéristiques EXT_SOURCE ont des corrélations négatives avec la cible, ce qui indique qu'à mesure que la valeur de EXT_SOURCE augmente, le client est plus susceptible de rembourser le prêt. Nous pouvons également voir que DAYS_BIRTH est positivement corrélé avec EXT_SOURCE_1 indiquant que peut-être l'un des facteurs de ce score est l'âge du client

EXT_SOURCE_3 affiche la plus grande différence entre les valeurs de la cible, cette caractéristique a une certaine relation avec la probabilité qu'un demandeur rembourse un prêt. La relation n'est pas très forte

# Feature Engineering
CREDIT_INCOME_PERCENT: 
le pourcentage du montant du crédit par rapport au revenu d'un client

ANNUITY_INCOME_PERCENT: le pourcentage de l'annuité du prêt par rapport au revenu du client

CREDIT_TERM: la durée du paiement en mois (puisque l'annuité est le montant mensuel dû

DAYS_EMPLOYED_PERCENT: le pourcentage des jours employés par rapport à l'âge du client

In [ ]:
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()


app_train_domain['INCOME_TO_ANNUITY_RATIO'] = app_train_domain['AMT_INCOME_TOTAL'] / app_train_domain['AMT_ANNUITY']
app_train_domain['INCOME_TO_ANNUITY_RATIO_BY_AGE'] = app_train_domain['INCOME_TO_ANNUITY_RATIO'] * app_train_domain['DAYS_BIRTH']
app_train_domain['INCOME_PER_PERSON'] = app_train_domain['AMT_INCOME_TOTAL'] / app_train_domain['CNT_FAM_MEMBERS']
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']
app_train_domain['PAYMENT_RATE'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']


app_test_domain['INCOME_TO_ANNUITY_RATIO'] = app_test_domain['AMT_INCOME_TOTAL'] / app_test_domain['AMT_ANNUITY']
app_test_domain['INCOME_TO_ANNUITY_RATIO_BY_AGE'] = app_test_domain['INCOME_TO_ANNUITY_RATIO'] * app_test_domain['DAYS_BIRTH']
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']
app_test_domain['INCOME_PER_PERSON'] = app_test_domain['AMT_INCOME_TOTAL'] / app_test_domain['CNT_FAM_MEMBERS']
app_test_domain['PAYMENT_RATE'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']

plt.figure(figsize = (12, 20))
# iterate through the new features
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    
    # create a new subplot for each source
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')
    
    # matplotlib.pyplot.legend
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html
    plt.legend()
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

In [ ]:
# Sauvegarde des résultats
app_train_domain.to_csv('app_train_domain_no_enconded_featureengineering.csv', index=False)
app_test_domain.to_csv('app_test_domain_no_enconded_featureengineering.csv', index=False)


In [ ]:
# Vérifier si 'Target' est une colonne dans le DataFrame
if 'TARGET' in app_train_domain.columns:
    print("La variable 'Target' est présente dans le DataFrame.")
else:
    print("La variable 'Target' n'est pas présente dans le DataFrame.")

# Traitement des valeurs manquantes

In [ ]:
app_train_ = pd.read_csv('app_train_domain_no_enconded_featureengineering.csv', sep=',')
app_test_ = pd.read_csv('app_test_domain_no_enconded_featureengineering.csv', sep=',')


In [ ]:
# Vérifier si 'Target' est une colonne dans le DataFrame
if 'TARGET' in app_train_.columns:
    print("La variable 'Target' est présente dans le DataFrame.")
else:
    print("La variable 'Target' n'est pas présente dans le DataFrame.")

In [ ]:
missing_values_table(app_train_)

### Suppression des valeurs manquantes de plus 90%

In [ ]:
# suppression des colonnes avec plus de 90% de valeurs manquantes pour les données d'entrainement
app_train_ = app_train_.loc[:, app_train_.isnull().mean() <.90]

In [ ]:
# Vérifier si 'Target' est une colonne dans le DataFrame
if 'TARGET' in app_train_.columns:
    print("La variable 'Target' est présente dans le DataFrame.")
else:
    print("La variable 'Target' n'est pas présente dans le DataFrame.")

### Encodage des variables catégorielles
Les variables catégorielles doivent être encodés pour être utilisables par les modèles. nous utiliserons Label Encoding pour toutes les variables catégorielles avec seulement 2 catégories et One-Hot Encoding pour toutes les variables catégorielles avec plus de 2 catégories

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Séparation des variables catégorielles et numériques
num_var = app_train_.select_dtypes(include=['int64', 'float64'])
cat_var = app_train_.select_dtypes(exclude=['int64', 'float64'])

# Encodage avec LabelEncoder pour les colonnes avec 2 catégories uniques
le = LabelEncoder()
le_count = 0

# Colonnes à encoder avec One-Hot Encoding
one_hot_columns = []

# Iterate through the categorical columns
for col in cat_var.columns:
    if len(app_train_[col].unique()) <= 2:
        # Appliquer le label encoder si 2 ou moins catégories
        app_train_[col] = le.fit_transform(app_train_[col])
        app_test_[col] = le.transform(app_test_[col])
        le_count += 1
    else:
        # Ajouter à la liste pour one-hot encoding si plus de 2 catégories
        one_hot_columns.append(col)

print(f'{le_count} columns were label encoded.')

# Appliquer One-Hot Encoding sur les colonnes restantes
app_train_ = pd.get_dummies(app_train_, columns=one_hot_columns, dummy_na=True)
app_test_ = pd.get_dummies(app_test_, columns=one_hot_columns, dummy_na=True)

# Assurer que les colonnes sont les mêmes entre les jeux d'entraînement et de test
app_train_, app_test_ = app_train_.align(app_test_, join='inner', axis=1)

# Affichage des trois premières lignes
app_train_.head(3)
print('Final Training Features shape: ', app_train_.shape)
print('Final Testing Features shape: ', app_test_.shape)


In [ ]:
# Sauvegarde des données
app_train_.to_csv('app_train_encoded.csv', index=False)
app_test_.to_csv('app_test_encoded.csv', index=False)


In [ ]:
# Vérifier si 'Target' est une colonne dans le DataFrame
if 'TARGET' in app_train_.columns:
    print("La variable 'Target' est présente dans le DataFrame.")
else:
    print("La variable 'Target' n'est pas présente dans le DataFrame.")

## Preprocessing et Fusion des tables
Les procédures de fusion des tables ci-dessous sont basés sur le pipeline de feature engineering présenté dans :

https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script

In [ ]:
import time

class Timer:
    """ Context manager for timing code execution """
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.start = time.time()
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.end = time.time()
        print(f"{self.name} - {self.end - self.start:.2f} s")


In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import time

def one_hot_encoder(df, nan_as_category=True):
    """ One-hot encode categorical features and handle missing values. """
    cat_cols = [c for c in df.columns if df[c].dtype == 'object']
    if nan_as_category:
        for c in cat_cols:
            df[c] = df[c].astype(str).fillna('NAN')
    df = pd.get_dummies(df, columns=cat_cols)
    return df, [col for col in df.columns if col.startswith(tuple(cat_cols))]

def bureau_and_balance(num_rows=None, nan_as_category=True):
    """ Preprocess bureau.csv and bureau_balance.csv """
    bureau = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\bureau.csv', nrows=num_rows)
    print(f'Read bureau samples: {len(bureau)} rows, {len(bureau.columns)} columns')
    
    bureau_balance = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\bureau_balance.csv', nrows=num_rows)
    print(f'Read bureau balance samples: {len(bureau_balance)} rows, {len(bureau_balance.columns)} columns')

    bureau_balance, bureau_balance_cat = one_hot_encoder(bureau_balance, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)

    # Perform aggregations and merge with bureau.csv
    bureau_balance_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bureau_balance_cat:
        bureau_balance_aggregations[col] = ['mean']
    
    bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bureau_balance_aggregations)
    bureau_balance_agg.columns = pd.Index([f"{e[0]}_{e[1].upper()}" for e in bureau_balance_agg.columns.tolist()])
    
    bureau = bureau.join(bureau_balance_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    
    del bureau_balance, bureau_balance_agg
    gc.collect()

    # Numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }

    # Categorical features
    cat_aggregations = {}
    for cat in bureau_cat:
        cat_aggregations[cat] = ['mean']
    for cat in bureau_balance_cat:
        cat_aggregations[f"{cat}_MEAN"] = ['mean']

    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index([f"BURO_{e[0]}_{e[1].upper()}" for e in bureau_agg.columns.tolist()])

    # Active and Closed credits
    for status in ['Active', 'Closed']:
        subset = bureau[bureau[f'CREDIT_ACTIVE_{status}'] == 1]
        subset_agg = subset.groupby('SK_ID_CURR').agg(num_aggregations)
        subset_agg.columns = pd.Index([f"{status.upper()}_{e[0]}_{e[1].upper()}" for e in subset_agg.columns.tolist()])
        bureau_agg = bureau_agg.join(subset_agg, how='left', on='SK_ID_CURR')
        del subset, subset_agg
    
    del bureau
    gc.collect()
    return bureau_agg

def previous_applications(num_rows=None, nan_as_category=True):
    """ Preprocess previous_application.csv """
    app_previous = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\previous_application.csv', nrows=num_rows)
    print(f'Read previous application samples: {len(app_previous)} rows, {len(app_previous.columns)} columns')
    
    app_previous, cat_cols = one_hot_encoder(app_previous, nan_as_category=True)
    
    # Handle specific missing value replacement
    for col in ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']:
        app_previous[col].replace(365243, np.nan, inplace=True)
    
    # Add feature
    app_previous['APP_CREDIT_PERC'] = app_previous['AMT_APPLICATION'] / app_previous['AMT_CREDIT']
    
    # Numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum']
    }

    # Categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    app_previous_agg = app_previous.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    app_previous_agg.columns = pd.Index([f"PREV_{e[0]}_{e[1].upper()}" for e in app_previous_agg.columns.tolist()])

    # Approved and Refused applications
    for status in ['Approved', 'Refused']:
        subset = app_previous[app_previous[f'NAME_CONTRACT_STATUS_{status}'] == 1]
        subset_agg = subset.groupby('SK_ID_CURR').agg(num_aggregations)
        subset_agg.columns = pd.Index([f"{status.upper()}_{e[0]}_{e[1].upper()}" for e in subset_agg.columns.tolist()])
        app_previous_agg = app_previous_agg.join(subset_agg, how='left', on='SK_ID_CURR')
    
    del app_previous
    gc.collect()
    return app_previous_agg

def pos_cash(num_rows=None, nan_as_category=True):
    """ Preprocess POS_CASH_balance.csv """
    pos = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\POS_CASH_balance.csv', nrows=num_rows)
    print(f'Read POS cash balance samples: {len(pos)} rows, {len(pos.columns)} columns')
    
    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)

    # Aggregations
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']

    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index([f"POS_{e[0]}_{e[1].upper()}" for e in pos_agg.columns.tolist()])
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    del pos
    gc.collect()
    return pos_agg

def installments_payments(num_rows=None, nan_as_category=True):
    """ Preprocess installments_payments.csv """
    ins = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\installments_payments.csv', nrows=num_rows)
    print(f'Read installments payments samples: {len(ins)} rows, {len(ins.columns)} columns')
    
    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)

    # Feature Engineering
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: max(x, 0))
    ins['DBD'] = ins['DBD'].apply(lambda x: max(x, 0))

    # Aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']

    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index([f"INSTAL_{e[0]}_{e[1].upper()}" for e in ins_agg.columns.tolist()])
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    del ins
    gc.collect()
    return ins_agg

def credit_card_balance(num_rows=None, nan_as_category=True):
    """ Preprocess credit_card_balance.csv """
    cc = pd.read_csv(r'C:\\Users\\saidi\\Projet_7\\Data\\credit_card_balance.csv', nrows=num_rows)
    print(f'Read credit card balance samples: {len(cc)} rows, {len(cc.columns)} columns')
    
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)
    
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index([f"CC_{e[0]}_{e[1].upper()}" for e in cc_agg.columns.tolist()])
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    
    del cc
    gc.collect()
    return cc_agg

def preprocess_pipeline(num_rows=None, debug=False):
    """ Main preprocessing pipeline """
    df = pd.read_csv('app_train_encoded.csv')  # Replace with actual path
    start = time.time()

    with Timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()

    with Timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()

    with Timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()

    with Timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()

    with Timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()

    time_taken = time.time() - start
    print(f'Preprocessing completed in {time_taken:.0f} s')
    return df

# File names and paths
RAW_DATA_FILENAME = 'HomeCredit_columns_description.csv'
CLEAN_DATA_FILENAME = 'cleaned_data_scoring.csv'
CLEAN_DATA_SAMPLE = 'cleaned_data_sample.csv'  # 100,000 records
CLEAN_DATA_FEATURES = 'cleaned_data_features.csv'  # Top 100 features
SAMPLE_SIZE = 10000

savepath = CLEAN_DATA_FILENAME

if os.path.exists(savepath):
    print('Reloading preprocessed_data (preprocess pipeline has not been modified since last edit)')
    df_data = pd.read_csv(savepath)
else:
    df_data = preprocess_pipeline(num_rows=None, debug=False)
    df_data.to_csv(savepath, index=False)

print(f'df_data.shape = {df_data.shape}')


In [ ]:
import numpy as np
import pandas as pd

def get_columns_with_infinite_values(df):
    """ Detect columns with infinite values and count them. """
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Find columns with infinite values
    inf_cols = numeric_df.columns[np.isinf(numeric_df).any()]
    
    res = []
    for col in inf_cols:
        inf_rows = np.isinf(numeric_df[col]).sum()
        res.append({'colonne': col, 'inf_count': inf_rows})
    return pd.DataFrame(res)

def replace_infinite_values(df):
    """ Replace infinite values with NaN. """
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Replace positive and negative infinity with NaN in numeric columns
    df[numeric_df.columns] = numeric_df.replace([np.inf, -np.inf], np.nan)
    
    return df

# Get columns with infinite values
inf_summary = get_columns_with_infinite_values(df_data)
print("Columns with infinite values:")
print(inf_summary)

# Replace infinite values with NaN
df_data_cleaned = replace_infinite_values(df_data)

# Optionally, save the cleaned DataFrame
# df_data_cleaned.to_csv('cleaned_data_with_nan.csv', index=False)

# Verify the replacement
inf_summary_after = get_columns_with_infinite_values(df_data_cleaned)
print("Columns with infinite values after replacement:")
print(inf_summary_after)
print(f'les valeurs infinite ont bien remplacé')

In [ ]:
df_data.head(3)

On voit qu'il y a les colonnes avec seulement une valeur et puisque l'imputation pour les modèles donne une variable qui est constante et le coefficient de correlation sera NaN, donc on va les supprimer 

In [ ]:
def get_single_value_cols(df: pd.DataFrame):
    unique_vals = df.nunique()
    return unique_vals[unique_vals == 1].index.to_list()

def drop_single_value_cols(df: pd.DataFrame, cols_to_drop=None):
    if cols_to_drop is None:
        cols_to_drop = get_single_value_cols(df)
    if len(cols_to_drop) > 0:
        print(f'supprimer les colonnes avec seulement une valeur, nb.cols = {len(cols_to_drop)}')
        return df.drop(columns=cols_to_drop)
    return df

df_data = drop_single_value_cols(df_data)
df_data.shape

In [ ]:
df_data.to_csv('data_clean.csv', index=False)
app_train_domain.to_csv('data_train.csv', index=False)
app_test_domain.to_csv('data_test.csv', index=False)
     

In [ ]:
# Vérifier si 'Target' est une colonne dans le DataFrame
if 'Target' in df_data.columns:
    print("La variable 'Target' est présente dans le DataFrame.")
else:
    print("La variable 'Target' n'est pas présente dans le DataFrame.")